In [1]:
import pandas as pd
pd.options.display.max_colwidth=100
import re, nltk, string
import numpy as np
import difflib
import re
import regex
import sys
from time import time
import matplotlib.pyplot as plt

ImportError: No module named 'regex'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from shallowlearn.models import FastText
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import clone
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.externals import joblib
from sklearn.svm import LinearSVC
from difflib import SequenceMatcher
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
#from IRModule import CosSimPredictor
from sklearn import preprocessing 
from sklearn.svm import SVC

In [ ]:
from scipy.sparse import coo_matrix, hstack, vstack
from nltk import ngrams
import sklearn_crfsuite
import pycrfsuite
import spacy
from spacy import displacy
import ipywidgets as widgets
import qgrid

In [4]:
scen_df = pd.read_csv('data/scenario_ds.txt')

In [14]:
prod_table=pd.read_excel('data/prod_dict.xlsx', sheet_name=0, dtype=str)

In [6]:
prod_table=prod_table.replace('nan', '', regex=True)
grid=qgrid.QGridWidget(df=prod_table,show_toolbar=True)
grid

QgridWidget(grid_options={'minVisibleRows': 8, 'rowHeight': 28, 'defaultColumnWidth': 150, 'explicitInitialization': True, 'enableTextSelectionOnCells': True, 'fullWidthRows': True, 'enableColumnReorder': False, 'forceFitColumns': True, 'syncColumnCellResize': True, 'editable': True, 'maxVisibleRows': 15, 'autoEdit': False}, precision=5, show_toolbar=True)

In [7]:
prod_table=grid.get_changed_df()

In [11]:
prod_table.loc[0].values

array(['djuice', 'one plan', ''], dtype=object)

In [8]:
txt_all=scen_df.Roman.str.cat()
all_docs=scen_df.English.append(scen_df.Roman)

In [25]:
all_docs=all_docs.str.replace('\n',' ')

In [9]:
def init_doc(s):
    return {'text':s, 'ents':[], 'title':None, }
    

In [10]:
def init(SNo=0):
    pre,inf,suf=prod_table.iloc[SNo,]
    #print (pre,inf,suf)
    pe=int(len(pre)/3); ie=int(len(inf)/3); se=int(len(suf)/3);
    return SNo,pe,ie,se,pre,inf,suf

In [26]:
def apply_regex(SNo,pe,ie,se,pre,inf,suf):
    docs=[]; pre_dct=set(); inf_dct=set(); suf_dct=set()
    regx=regex.compile(r'(?e:)(?:(%s){e<=%d}) (%s){e<=%d} (?:(%s){e<=%d})'%(pre,pe,inf,ie,suf,se), regex.I)
    for i in all_docs:
        doc=init_doc(i)
        match = regx.finditer(i)
        for m in match:
            tmp=[{'start': m.start(), 'end': m.end(), 'label': 'ENT'}]
            pre_dct.add(m.groups()[0])
            inf_dct.add(m.groups()[1])
            suf_dct.add(m.groups()[2])
            doc['ents']+=tmp
        if doc['ents']!=[]:
            docs.append(doc)   
    return docs, pre_dct, inf_dct, suf_dct

In [27]:
SNo,pe,ie,se,pre,inf,suf = init(SNo=7)
docs, pre_dct, inf_dct, suf_dct = apply_regex(SNo,pe,ie,se,pre,inf,suf)

In [28]:
from ipywidgets import Button, HBox, VBox, Box, Text, Layout
from ipywidgets import interact, interactive, fixed

In [29]:
box_layout=Layout(
    display='flex',
    flex_flow='row',
    border='solid 1px',
    align_items='stretch',
    width='75%',
)

html_box_layout=Layout(
    overflow_x='scroll',
    display='flex',
    flex_flow='row',
    height='100 px',
    border='solid',
    
)

In [30]:
p,i,s=prod_table.iloc[SNo,]

In [31]:
pre_chk=widgets.Checkbox(value=True,)
inf_chk=widgets.Checkbox(value=True,)
suf_chk=widgets.Checkbox(value=True,)

pre_sel=widgets.SelectMultiple(options=pre_dct)
inf_sel=widgets.SelectMultiple(options=inf_dct)
suf_sel=widgets.SelectMultiple(options=suf_dct)

pre_txt=Text(value=p, disabled=not pre_chk.value)
inf_txt=Text(value=i, disabled=not inf_chk.value)
suf_txt=Text(value=s, disabled=not suf_chk.value)

pre_sld=widgets.IntSlider(value=pe,min=0, max=6, orientation='vertical',readout=True,)
inf_sld=widgets.IntSlider(value=ie,min=0, max=6, orientation='vertical',readout=True,)
suf_sld=widgets.IntSlider(value=se,min=0, max=6, orientation='vertical',readout=True,)

pre_txt_bx=VBox([pre_txt, pre_sel])
inf_txt_bx=VBox([inf_txt, inf_sel])
suf_txt_bx=VBox([suf_txt, suf_sel])

pre_box=HBox([pre_chk,pre_txt_bx,pre_sld])
inf_box=HBox([inf_chk,inf_txt_bx,inf_sld])
suf_box=HBox([suf_chk,suf_txt_bx,suf_sld])

w=widgets.IntText(SNo,description='Serial No:')
prev_btn=widgets.Button(description="Previous")
next_btn=widgets.Button(description="Next")

submit_btn=widgets.Button(description='Submit')
update_btn=widgets.Button(description='Update')

btn_layer1=HBox([w, prev_btn, next_btn])
btn_layer2=HBox([submit_btn,update_btn])

In [32]:
def bind():
    SNo=w.value; 
    pe=pre_sld.value; ie=inf_sld.value; se= suf_sld.value
    pre=pre_txt.value if pre_chk.value else '' 
    inf=inf_txt.value if inf_chk.value else ''
    suf=suf_txt.value if suf_chk.value else ''
    return SNo,pe,ie,se,pre,inf,suf
    

In [33]:
def bind_widg(SNo,pe,ie,se,pre,inf,suf,pdct,idct,sdct):
    w.value=SNo; 
    pre_sld.value=pe; inf_sld.value=ie; suf_sld.value=se
    pre_txt.value=pre; pre_chk.value=True
    inf_txt.value=inf; inf_chk.value=True
    suf_txt.value=suf; suf_chk.value=True
    pre_sel.options=pdct;inf_sel.options=idct;suf_sel.options=sdct
    

In [34]:
def on_submit_clicked(b):
    SNo,pe,ie,se,pre,inf,suf = bind()
    docs, pre_dct, inf_dct, suf_dct=apply_regex(SNo,pe,ie,se,pre,inf,suf)
    w_html.value=displacy.render(docs, style='ent', manual=True,)

submit_btn.on_click(on_submit_clicked)

In [35]:
def on_prev_clicked(b):
    SNo,pe,ie,se,pre,inf,suf = bind()
    SNo=SNo-1
    SNo,pe,ie,se,pre,inf,suf = init(max(SNo,0))
    docs, pre_dct, inf_dct, suf_dct=apply_regex(SNo,pe,ie,se,pre,inf,suf)
    bind_widg(SNo,pe,ie,se,pre,inf,suf,pre_dct, inf_dct, suf_dct)
    w_html.value=displacy.render(docs, style='ent', manual=True,)

prev_btn.on_click(on_prev_clicked)

In [36]:
def on_next_clicked(b):
    SNo,pe,ie,se,pre,inf,suf = bind()
    SNo=SNo+1
    SNo,pe,ie,se,pre,inf,suf = init(min(SNo,len(prod_table)-1))
    docs, pre_dct, inf_dct, suf_dct=apply_regex(SNo,pe,ie,se,pre,inf,suf)
    bind_widg(SNo,pe,ie,se,pre,inf,suf,pre_dct, inf_dct, suf_dct)
    w_html.value=displacy.render(docs, style='ent', manual=True,)

next_btn.on_click(on_next_clicked)

In [37]:
w_html=widgets.HTML(displacy.render(docs, style='ent', manual=True,))
#items = [Text(value=w, disabled=True) for w in prod_table.iloc[SNo,]]
item_bx=Box([pre_box,inf_box,suf_box], )
wBox=widgets.Box([w_html], layout=html_box_layout)

VBox([btn_layer1, item_bx, btn_layer2, wBox, HBox([prev_btn, next_btn])])

VBox(children=(HBox(children=(IntText(value=7, description='Serial No:'), Button(description='Previous', style=ButtonStyle()), Button(description='Next', style=ButtonStyle()))), Box(children=(HBox(children=(Checkbox(value=True), VBox(children=(Text(value='talkshawk'), SelectMultiple(options=('Talkshawk',), value=()))), IntSlider(value=3, max=6, orientation='vertical'))), HBox(children=(Checkbox(value=True), VBox(children=(Text(value='econmy'), SelectMultiple(options=('Economy.', 'Economy,', 'Economy'), value=()))), IntSlider(value=2, max=6, orientation='vertical'))), HBox(children=(Checkbox(value=True), VBox(children=(Text(value=''), SelectMultiple(options=('',), value=()))), IntSlider(value=0, max=6, orientation='vertical'))))), HBox(children=(Button(description='Submit', style=ButtonStyle()), Button(description='Update', style=ButtonStyle()))), Box(children=(HTML(value='<div class="entities" style="line-height: 2.5">Super Hit Offer:  Best call offer in Talkshawk packages is Super Hit Offer ,Dial *345*011# and make onnet free calls all day except 7 pm to 10 pm in Rs.15 Incl. Tax. Applicable on \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Talkshawk Economy \n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ENT</span>\n</mark>\nPlus, Talkshawk A1, TS Econmy Plus, TS Economy, TS 24 Hour ,TS 30 & TS Value package. Validity 1 day.   There are call setup charges of 15 paisa incl. tax for all on-net & off-net charged calls, discounted calls, free calls, FNF calls, & IDD calls.</div>\n\n<div class="entities" style="line-height: 2.5">Mini Budget Daily:  Dial *240# and get 50 On-net Minutes, 300 SMS & 3 MB Internet in Rs.15 Incl. Tax. Applicable on \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Talkshawk Economy \n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ENT</span>\n</mark>\nPlus, Talkshawk A1, TS Economy, TS 24, TS 30 & TS Value. Validity 1 day.    There are call setup charges of 15 paisa incl. tax for all on-net & off-net charged calls, discounted calls, free calls, FNF calls, & IDD calls.</div>\n\n<div class="entities" style="line-height: 2.5">Mini Budget Offer-2: To subscribe dial *050# in Rs.18 Incl. Tax and get 100 On-net Minutes, 300 SMS & 3 MB Internet. Applicable on \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Talkshawk Economy \n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ENT</span>\n</mark>\nPlus, Talkshawk A1, TS Economy, TS 24, TS 30 & TS Value. Validity 1 day.   There are call setup charges of 15 paisa incl. tax for all on-net & off-net charged calls, discounted calls, free calls, FNF calls, & IDD calls.</div>\n\n<div class="entities" style="line-height: 2.5">Talkshawk Voice Offer: To subscribe dial *001# in Rs.17.91 Incl.Tax and make on-net free calls all day except 7pm to 10 pm. Offer valid for a day. This offer is applicable on Talkshawk 24 Hours, Talkshawk Value and \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Talkshawk Economy. \n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-

In [ ]:
pd.